In [1]:
!pip install vina mdanalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.8 MB/s eta 0:00:00


# **Files preparation**



In [1]:
import requests

# Download the protein file
pdb_id = '1lpb'

protein_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
print(f"{pdb_id} structure is downloaded")

# Send the request and save the returned JSON blob as a variable
protein_request = requests.get(protein_url)
protein_request.raise_for_status() # Check for errors

1lpb structure is downloaded


In [2]:
# Define the full path to the file you want to save
protein_filepath = f"/content/{pdb_id}.pdb"

# Save the actual text of the returned PDB blob as a file
with open(protein_filepath, "w") as f:
    f.write(protein_request.text)

print(f"Saved protein to {protein_filepath}")

Saved protein to /content/1lpb.pdb


# **Docking w/ Catalitic Triad's Coordinate**

In [3]:
import MDAnalysis as mda

protein = mda.Universe('/content/1lpb.pdb')
ligand_select = protein.select_atoms('resid 451') # There are more than 1 ligand, so we have to specify the location

box_size = ligand_select.positions.max(axis=0) - ligand_select.positions.min(axis=0) + 5
print(box_size)

[16.792    13.950999 13.709   ]


In [4]:
# Use the box size and convert them to list
box_size = box_size.tolist()

In [ ]:
# Set up the box center (by using the catalytic triad's coordinates)
pocket_center = [4.641838, 28.72172, 49.18972]
print(pocket_center)

[4.641838, 28.72172, 49.18972]


In [ ]:
# Selecting the scoring function method
from vina import Vina
v = Vina(sf_name='vina')

# Defining the molecules
v.set_receptor('/content/1LPB_restrain_xr.pdbqt')
v.set_ligand_from_file('/content/BOG_ideal.pdbqt')
v.compute_vina_maps(center=pocket_center, box_size=box_size)

Running the docking process

In [ ]:
v.dock(exhaustiveness=32, n_poses=5)

In [ ]:
v.write_poses('BOG_cat.pdbqt', n_poses=5, overwrite=True)

In [ ]:
v.energies()

array([[-3.093, -5.173, -0.137,  2.08 , -0.137],
       [-2.519, -4.607,  0.258,  1.693, -0.137],
       [-1.957, -3.477,  0.066,  1.316, -0.137],
       [-1.796, -4.013,  0.871,  1.208, -0.137],
       [-1.58 , -2.89 ,  0.11 ,  1.063, -0.137]])

In [ ]:
import pandas as pd

# Adding columns name
column_names = ['total', 'inter', 'intra', 'torsions', 'intra best pose']

df = pd.DataFrame(v.energies(), columns=column_names)
df.head()

,total,inter,intra,torsions,intra best pose
0,-3.093,-5.173,-0.137,2.080,-0.137
1,-2.519,-4.607,0.258,1.693,-0.137
2,-1.957,-3.477,0.066,1.316,-0.137
3,-1.796,-4.013,0.871,1.208,-0.137
4,-1.580,-2.890,0.110,1.063,-0.137


# **Use the native BOG position**

In [5]:
# Define the docking area based on native ligand
native = ligand_select.center_of_geometry()
print(native)

[ 4.42795833 15.4547708  47.4288125 ]


In [6]:
native = native.tolist()

In [ ]:
# Selecting the scoring function method
from vina import Vina
v = Vina(sf_name='vina')

# Defining the molecules
v.set_receptor('/content/1LPB_restrain_xr.pdbqt')
v.set_ligand_from_file('/content/BOG_ideal.pdbqt')
v.compute_vina_maps(center=native, box_size=box_size)

Running the Docking Process

In [ ]:
v.dock(exhaustiveness=32, n_poses=5)

In [ ]:
v.write_poses('BOG_native.pdbqt', n_poses=5, overwrite=True)

In [ ]:
v.energies()

array([[-5.583, -9.336, -1.182,  3.753, -1.182],
       [-5.498, -9.748, -0.628,  3.696, -1.182],
       [-5.454, -9.505, -0.798,  3.667, -1.182],
       [-5.441, -9.324, -0.958,  3.658, -1.182],
       [-5.315, -9.172, -0.899,  3.574, -1.182]])

In [ ]:
import pandas as pd

# Adding columns name
column_names = ['total', 'inter', 'intra', 'torsions', 'intra best pose']

df = pd.DataFrame(v.energies(), columns=column_names)
df.head()

,total,inter,intra,torsions,intra best pose
0,-5.583,-9.336,-1.182,3.753,-1.182
1,-5.498,-9.748,-0.628,3.696,-1.182
2,-5.454,-9.505,-0.798,3.667,-1.182
3,-5.441,-9.324,-0.958,3.658,-1.182
4,-5.315,-9.172,-0.899,3.574,-1.182


# **Actual Docking**

In [7]:
# Define the grid box size and coordinate, as we see, the native dock area has better binding score
dock_center = native
dock_box_size = box_size

# For Single Ligand

In [ ]:
import os
from vina import Vina
import pandas as pd

# Selecting the scoring function method
from vina import Vina
v = Vina(sf_name='vina')

# Defining the molecules
v.set_receptor('/content/1lpb_protein.pdbqt')
ligand_name = 'orlistat'
v.set_ligand_from_file(f'/content/prepared_ligands/{ligand_name}.pdbqt')
v.compute_vina_maps(center=native, box_size=box_size)

v.dock(exhaustiveness=64, n_poses=5)

v.write_poses(f'/content/docking_results/{ligand_name}', n_poses=1, overwrite=True)

In [ ]:
v.energies()

In [ ]:
import pandas as pd

# Adding columns name
column_names = ['total', 'inter', 'intra', 'torsions', 'intra best pose']

df = pd.DataFrame(v.energies(), columns=column_names)
df.head()

In [ ]:
df.to_csv(f'/content/csv/{ligand_name}.csv', index=False)

# For Multi Ligands

In [8]:
import os
from vina import Vina
import pandas as pd

# --- Configuration Paths ---
# Directory containing your prepared ligand .pdbqt files
LIGANDS_INPUT_FOLDER = '/content/prepared_ligands/'
# Directory where the docked pose .pdbqt files will be saved
DOCKING_POSES_OUTPUT_FOLDER = '/content/docking_poses/'
# Directory where individual ligand energy CSV files will be saved
ENERGY_CSVS_OUTPUT_FOLDER = '/content/energy_csvs/'
# Path to your receptor .pdbqt file
RECEPTOR_PATH = '/content/1lpb_protein.pdbqt'

# --- Ensure Output Directories Exist ---
os.makedirs(DOCKING_POSES_OUTPUT_FOLDER, exist_ok=True)
os.makedirs(ENERGY_CSVS_OUTPUT_FOLDER, exist_ok=True)

# --- Vina Initialization ---
print("Initializing Vina...")
v = Vina(sf_name='vina')

# Set the receptor for docking
try:
    v.set_receptor(RECEPTOR_PATH)
    print(f"Receptor set: {RECEPTOR_PATH}")
except Exception as e:
    print(f"Error setting receptor from {RECEPTOR_PATH}: {e}")
    print("Please ensure the receptor file exists and is correctly formatted.")
    exit() # Exit if receptor cannot be set

print(f"Computing Vina maps with center: {native} and box_size: {box_size}")
try:
    v.compute_vina_maps(center=native, box_size=box_size)
    print("Vina maps computed successfully.")
except Exception as e:
    print(f"Error computing Vina maps: {e}")
    print("Please check your 'native' coordinates and 'box_size' values.")
    exit() # Exit if maps cannot be computed

# --- Prepare to Process Ligands ---
# Define column names for the energy CSVs
column_names = ['total_energy', 'inter_energy', 'intra_energy', 'torsions_energy', 'intra_best_pose_energy']

# Get list of all .pdbqt ligand files
ligand_files = [f for f in os.listdir(LIGANDS_INPUT_FOLDER) if f.endswith('.pdbqt')]
print(f"\nFound {len(ligand_files)} .pdbqt ligand files in '{LIGANDS_INPUT_FOLDER}'.")

if not ligand_files:
    print(f"No .pdbqt ligand files found. Please ensure your ligands are in '{LIGANDS_INPUT_FOLDER}'.")
else:
    # --- Iterate Through Each Ligand ---
    for ligand_file in ligand_files:
        ligand_name = os.path.splitext(ligand_file)[0] # Get ligand name without extension
        ligand_path = os.path.join(LIGANDS_INPUT_FOLDER, ligand_file)

        print(f"\n--- Processing Ligand: {ligand_name} ---")

        try:
            # Set the current ligand for docking
            v.set_ligand_from_file(ligand_path)
            print(f"Ligand '{ligand_name}' loaded.")

            # Perform docking
            print("Starting docking process...")
            v.dock(exhaustiveness=64, n_poses=5) # exhaustiveness and n_poses as per your original request
            print("Docking complete.")

            # --- Save Docked Pose ---
            docked_pose_output_path = os.path.join(DOCKING_POSES_OUTPUT_FOLDER, f'{ligand_name}_docked.pdbqt')
            v.write_poses(docked_pose_output_path, n_poses=1, overwrite=True) # Save only the best pose
            print(f"Top docked pose saved to: {docked_pose_output_path}")

            # --- Get and Save Energy Results ---
            energies_raw = v.energies()

            # Convert raw energies to a list for consistent processing
            energies = []
            if energies_raw is not None:
                try:
                    energies = list(energies_raw)
                except TypeError:
                    print(f"Warning: v.energies() for {ligand_name} returned an unexpected type ({type(energies_raw)}). Skipping energy CSV for this ligand.")

            if energies and len(energies) > 0:
                best_pose_energies = energies[0] # Get energies for the best pose

                # Convert all energy values to float, using np.nan for any unconvertible values
                # This ensures the DataFrame will always have numeric columns
                processed_energies = []
                for val in best_pose_energies:
                    try:
                        processed_energies.append(float(val))
                    except (ValueError, TypeError):
                        processed_energies.append(np.nan) # Use np.nan for problematic values

                # Ensure we have exactly 5 energy values for the DataFrame, padding with np.nan if needed
                while len(processed_energies) < 5:
                    processed_energies.append(np.nan)

                # Create a DataFrame for the current ligand's energies
                # We slice to ensure only the first 5 components are used if more are returned by v.energies()
                df_ligand_energy = pd.DataFrame([processed_energies[:5]], columns=column_names)

                # Save the DataFrame to a specific CSV file for this ligand
                energy_csv_output_path = os.path.join(ENERGY_CSVS_OUTPUT_FOLDER, f'{ligand_name}_energies.csv')
                df_ligand_energy.to_csv(energy_csv_output_path, index=False)
                print(f"Energies for '{ligand_name}' saved to: {energy_csv_output_path}")
                print(f"Energies data:\n{df_ligand_energy.head()}")
            else:
                print(f"No valid energies retrieved for '{ligand_name}'. No energy CSV will be generated for this ligand.")

        except Exception as e:
            print(f"An unexpected error occurred while processing '{ligand_name}': {e}")
            print(f"Skipping '{ligand_name}' due to error.")

print("\n--- All ligands processed. Script finished. ---")

Initializing Vina...
Receptor set: /content/1lpb_protein.pdbqt
Computing Vina maps with center: [4.427958327888821, 15.454770803451538, 47.4288125038147] and box_size: [16.79199981689453, 13.95099925994873, 13.708999633789062]
Vina maps computed successfully.

Found 11 .pdbqt ligand files in '/content/prepared_ligands/'.

--- Processing Ligand: Xanthotoxol ---
Ligand 'Xanthotoxol' loaded.
Starting docking process...
Docking complete.
Top docked pose saved to: /content/docking_poses/Xanthotoxol_docked.pdbqt
Energies for 'Xanthotoxol' saved to: /content/energy_csvs/Xanthotoxol_energies.csv
Energies data:
   total_energy  inter_energy  intra_energy  torsions_energy  \
0        -6.823        -7.023           0.0            0.199   

   intra_best_pose_energy  
0                     0.0  

--- Processing Ligand: Quinolone ---
Ligand 'Quinolone' loaded.
Starting docking process...
Docking complete.
Top docked pose saved to: /content/docking_poses/Quinolone_docked.pdbqt
Energies for 'Quinolon

# **Results Handling**

In [12]:
import os
import pandas as pd
from io import StringIO

def merge_first_row_csvs_from_folder(directory_path):
    """
    Merges the first row of data from multiple CSV files found in a specified directory,
    extracting the ligand name from each filename and adding it as a new column.
    It also ensures the 'ligand_name' column is the last column and removes
    the '_energies' suffix from the ligand names.

    Args:
        directory_path (str): The path to the directory containing the CSV files.

    Returns:
        pandas.DataFrame: A DataFrame containing the merged first rows
                          with an added 'ligand_name' column.
    """
    all_data = []

    if not os.path.isdir(directory_path):
        print(f"Error: Directory '{directory_path}' not found.")
        return pd.DataFrame()

    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            filepath = os.path.join(directory_path, filename)
            try:
                # Extract ligand name from the filename
                ligand_name = os.path.splitext(filename)[0]

                # Remove the '_energies' suffix if present
                if ligand_name.endswith('_energies'):
                    ligand_name = ligand_name[:-len('_energies')]

                with open(filepath, 'r') as f:
                    content = f.read()

                # Use StringIO to treat the string content as a file
                # Read only the first row
                df = pd.read_csv(StringIO(content), nrows=1)

                # Add the 'ligand_name' column
                df['ligand_name'] = ligand_name

                # Explicitly reorder columns to ensure 'ligand_name' is the very last column
                # This creates a new list of columns with 'ligand_name' at the end
                cols = [col for col in df.columns if col != 'ligand_name'] + ['ligand_name']
                df = df[cols]

                # Append the DataFrame row to our list
                all_data.append(df)

            except pd.errors.EmptyDataError:
                print(f"Warning: File {filename} is empty or only contains headers. Adding ligand name with None values for energies.")
                # Extract ligand name and remove suffix for empty files too
                ligand_name = os.path.splitext(filename)[0]
                if ligand_name.endswith('_energies'):
                    ligand_name = ligand_name[:-len('_energies')]

                # Define the expected columns for consistency, including ligand_name
                # The energy columns will be filled with None (which pandas converts to NaN) and only the ligand_name will have a value.
                expected_columns = ['total_energy', 'inter_energy', 'intra_energy', 'torsions_energy', 'intra_best_pose_energy', 'ligand_name']
                df = pd.DataFrame(columns=expected_columns)
                df.loc[0] = [None] * (len(expected_columns) - 1) + [ligand_name]
                all_data.append(df)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue

    if not all_data:
        print("No CSV files found or processed in the specified directory.")
        return pd.DataFrame()

    # Concatenate all individual DataFrames into one
    merged_df = pd.concat(all_data, ignore_index=True)
    return merged_df

# Set path
csv_folder_path = '/content/energy_csvs'

merged_results = merge_first_row_csvs_from_folder(csv_folder_path)

# Print the merged DataFrame
print("Merged Results:")
print(merged_results)

# Save the DataFrame to a new CSV file
merged_results.to_csv('merged_ligand_data.csv', index=False)

Merged Results:
    total_energy  inter_energy  intra_energy  torsions_energy  \
0         -5.875       -13.774        12.984            7.899   
1         -7.633        -8.302        -0.088            0.669   
2         -5.367        -6.151        -0.125            0.784   
3         -4.428        -4.946        -0.074            0.518   
4         -5.871        -7.416        -0.375            1.545   
5         -5.213        -5.823        -0.164            0.610   
6         -6.823        -7.023         0.000            0.199   
7         -4.055        -4.292        -0.065            0.237   
8         -5.417        -5.417         0.000            0.000   
9         -4.442        -4.702        -0.015            0.260   
10        -4.798        -5.920        -0.224            1.122   

    intra_best_pose_energy          ligand_name  
0                   12.984             orlistat  
1                   -0.088            Oxycodone  
2                   -0.125           Tryptamine  
3  

**Download the result folders**

In [14]:
import shutil
shutil.make_archive('/content/docking_poses_1', 'zip', '/content/docking_poses')

from google.colab import files
files.download('/content/docking_poses_1.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
import shutil
shutil.make_archive('/content/energy_csv_1', 'zip', '/content/energy_csvs')

from google.colab import files
files.download('/content/energy_csv_1.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>